### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2026-01-23 18:14:25", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-01-23 18:14:25", 9927.604]], "total_portfolio_value": 9927.604, "total_profit_loss": -72.39600000000064}'

In [5]:
account.report()

'{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2026-01-23 18:14:25", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-01-23 18:14:25", 9927.604], ["2026-01-23 18:14:27", 9981.604]], "total_portfolio_value": 9981.604, "total_profit_loss": -18.39600000000064}'

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 66.132,
  'timestamp': '2026-01-23 18:14:25',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [7]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [8]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='buy_shares', 

In [9]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [10]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Ed, your current cash balance is $9,801.60. You are holding 3 shares of Amazon (AMZN) in your account. If you need any further information or want to make any transactions, please let me know!

### Now let's build our own MCP Client

In [11]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None), Tool(name='buy_shares', ti

In [12]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your account balance is $9,801.60. If you need any other information or want to make any transactions, please let me know!

In [13]:
context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2026-01-23 18:14:25", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-01-23 18:14:25", 9927.604], ["2026-01-23 18:14:27", 9981.604], ["2026-01-23 18:28:03", 9993.604]], "total_portfolio_value": 9993.604, "total_profit_loss": -6.39600000000064}


In [14]:
from accounts import Account
Account.get("ed").report()

'{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2026-01-23 18:14:25", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2026-01-23 18:14:25", 9927.604], ["2026-01-23 18:14:27", 9981.604], ["2026-01-23 18:28:03", 9993.604], ["2026-01-23 18:28:11", 9858.604]], "total_portfolio_value": 9858.604, "total_profit_loss": -141.39600000000064}'

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>

In [21]:
from agents.mcp import MCPServerStdio

params = {
    "command" : "python",
    "args" : ["date_server.py"]
}

async def test_server():
    async with MCPServerStdio(params=params) as server:
        tools = await server.list_tools()
        print(tools)

await test_server()

[Tool(name='get_current_date', title=None, description='"Return the current date.', inputSchema={'properties': {}, 'title': 'get_current_dateArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'get_current_dateOutput', 'type': 'object'}, icons=None, annotations=None, meta=None)]


In [24]:
from agents import Agent, Runner
from agents.mcp import MCPServerStdio

instructions = "You can call MCP tools to answer questions."

async def run_agent():
    async with MCPServerStdio(params=params) as server:
        
        agent = Agent(
            name="date_agent",
            instructions=instructions,
            model="gpt-4.1-mini",
            mcp_servers=[server],
        )

        result = await Runner.run(agent, "What is today's date?")
        print(result.final_output)

await run_agent()

Today's date is January 23, 2026.
